# Clustering, Alignment and labels


## Import data and libraries


In [ ]:

# import sys
import os
# import scipy as sc
# from scipy import stats
import numpy as np
import pandas as pd
# import datetime as dt
import math
import pickle
import scipy.cluster.hierarchy as shc
from tqdm import tqdm
# import time
from Bio import AlignIO
import copy
# from spmf import Spmf
# from Bio.Align.Applications import MafftCommandline as mafft
from levenstein_dp import levenshteinDistanceDP as ld
import editdistance


# sys.setrecursionlimit(5000000)


In [ ]:
# File export suffix
start_index = 0
number_of_stays = 1000
display_matrix = False
output_path = "output/"
run_test_data = True
output_folder = f"stays-{number_of_stays}/"
alignments_output = f"{output_folder}alignments/"
number_of_events_test = 20

# output_folder = f"stays-test-{number_of_stays}/"

# file_suffix = '_test_' + str(number_of_stays)
file_suffix = '_' + str(number_of_stays)


### Store and load function


In [ ]:
def save_as_pickle(data, file_name, path=output_path):
    file = open(path + file_name, 'wb')
    pickle.dump(data, file)
    file.close()


def get_pickle(file_name, path=output_path):
    return pickle.load(open(path + file_name, 'rb'))


### Store as fasta file


In [ ]:
def sequence_to_fasta(sequences: list, sequence_ids, id, path=output_path, folder=output_folder):
    if not os.path.isdir(path + folder):
        print('[INFO] creating folder')
        os.makedirs(path + folder)
        
    file = open(f"{path + folder}sequences-{id}.fa", 'w')
   
    print('[INFO] writing file...')
    for i in range(len(sequences)):
        file.write(f">{sequence_ids[i]}\n{sequences[i]}\n")
    file.close()
    print('[INFO] writing done')


### Load complete dataset


In [ ]:
data = get_pickle('data_complete_v4.1')
data.head()


### Load distance matrix dataset & set stays


In [ ]:
dist_data = get_pickle("distance_data_v4.1")


In [ ]:
# dist_data.sort_values(by='event_id')
# dist_data

In [ ]:
stays = list(dist_data['hadm_id'].unique())[
    start_index: start_index + number_of_stays]


## Create test dataset of limited length and \# stays


In [ ]:
if run_test_data:
    test_data = pd.DataFrame(columns=data.columns)
    test_data_dist = pd.DataFrame(columns=dist_data.columns)
    for stay in stays:
        events = data[data['hadm_id'] == stay]
        events_dist = dist_data[dist_data['hadm_id'] == stay]
        test_data = pd.concat(
            [test_data, events.head(n=number_of_events_test)])
        test_data_dist = pd.concat(
            [test_data_dist, events_dist.head(n=number_of_events_test)])

    test_data = test_data.reset_index()
    test_data_dist = test_data_dist.reset_index()
    test_data = test_data.drop(columns=['index'])
    test_data_dist = test_data_dist.drop(columns=['index'])
    test_data


In [ ]:
if run_test_data:
    data = test_data
    dist_data = test_data_dist
    save_as_pickle(data, 'events' + file_suffix + "_test")


## Compute distance matrix


In [ ]:
save_as_pickle(data, 'events' + file_suffix + "_test")


In [ ]:
# def compute_distance_matrix(number_of_stays=number_of_stays):
#     stays = list(dist_data['hadm_id'].unique())[
#         start_index: start_index + number_of_stays]
#     distances = []

#     for y in tqdm(range(len(stays))):
#         sequence_y = dist_data[dist_data['hadm_id']
#                                == stays[y]]['event_encoded'].tolist()
#         distance_row = []

#         for x in tqdm(range(y, len(stays))):
#             sequence_x = dist_data[dist_data['hadm_id']
#                                    == stays[x]]['event_encoded'].tolist()

#             # Diagonal
#             if stays[y] == stays[x]:
#                 distance_row.append(0)
#                 continue
#             # All other
#             else:
#                 max_length = max(len(sequence_x), len(sequence_y))
#                 distance_row.append(ld(sequence_y, sequence_x)/max_length)

#         distances.append(distance_row)

#     # print(distances)
#     if display_matrix:
#         print('Computed distance matrix:')

#         for line in distances:
#             print('  '.join(map(str, line)))
#     return distances


In [ ]:
def compute_distance_matrix():
    sequences = [dist_data[dist_data['hadm_id']
                           == hadm_id]['event_encoded'].tolist() for hadm_id in stays]

    print("[INFO] Data Loaded ")

    length = len(sequences)
    outputMatrix = [[0] * length for _i in range(length)]

    progress = 0
    updateStep = 100
    with tqdm(total=0.5*(length * length)) as pbar:
        for idxA in range(0, length):
            for idxB in range(idxA, length):
                max_length = max(len(sequences[idxA]), len(sequences[idxB]))
                distance = editdistance.eval(
                    sequences[idxA], sequences[idxB])/max_length
                outputMatrix[idxA][idxB] = distance
                outputMatrix[idxB][idxA] = distance
                if (progress % updateStep == 0):
                    pbar.update(updateStep)
                progress += 1
    
    return outputMatrix


In [ ]:
dist_matrix = compute_distance_matrix()
# save_as_pickle(dist_matrix, 'distance_matrix_test_' + str(number_of_stays))
save_as_pickle(dist_matrix, 'distance_matrix_' + str(number_of_stays))
save_as_pickle(dist_matrix, 'dist_matrix_' + str(number_of_stays))


In [ ]:
dist_matrix

## Hierarchical clustering


In [ ]:
# dist_matrix = get_pickle('distance_matrix_50')
# dist_matrix = get_pickle('distance_matrix_3')


In [ ]:
dist_matrix = get_pickle('distance_matrix_' + str(number_of_stays))


In [ ]:
# def get_sequence_distance_list(u, v):
#     index_u, index_v = stays.index(u[0]), stays.index(v[0])
#     return dist_matrix[min(index_u, index_v)][max(index_u, index_v) - min(index_u, index_v)]


In [ ]:
def get_sequence_distance_matrix(u, v):
    index_u, index_v = stays.index(u[0]), stays.index(v[0])
    return dist_matrix[index_u][index_v]


In [ ]:
clust_data = data.drop_duplicates(subset=['hadm_id'])[
    start_index: start_index + number_of_stays]

clust_data = clust_data.drop(columns=['event_id', 'subject_id', 'transfer_id', 'eventtype',
                                      'careunit', 'intime', 'outtime', 'charttime', 'event',
                                      'value', 'valuenum', 'valueuom',
                                      'label', 'category', 'param_type',
                                      'value_categorical',
                                      'event_encoded'])

# links = shc.linkage(clust_data, metric=get_sequence_distance_list)
links = shc.linkage(clust_data, metric=get_sequence_distance_matrix)
dend = shc.dendrogram(links, labels=stays, leaf_rotation=-90)


## Display test sequences


In [ ]:
sequences = []
for stay in stays:
    events = data[data['hadm_id'] == stay]
    # print(f"seq {stay}: {''.join(list(events['event_encoded_alphabet']))}")
    sequences.append(''.join(list(events['event_encoded'])))
    # print(f"seq {stay}: {''.join(list(events['event_encoded']))}")

# sequences


## Aligning sequences


In [ ]:
def sort_by_indexes(list_data, indexes, reverse=False):
  return [val for (_, val) in sorted(zip(indexes, list_data), key=lambda x:
          x[0], reverse=reverse)]
  
def get_clusters_by_level(level, links):
    return list(shc.fcluster(links, t=level, criterion="distance"))


def get_aggregated_sequence(al_seq):
    agg_sequence = list(
        zip(*[sequence.seq for sequence in al_seq]))
    # Remove duplicates
    agg_sequence = [list(set(agg_event)) for agg_event in agg_sequence]
    # Convert characters to numbers
    agg_sequence = [[event for event in agg_event]
                    for agg_event in agg_sequence]
    # agg_sequence = [[str(character_to_number(event))
    #                  for event in agg_event] for agg_event in agg_sequence]
    # Only have lists when aggregate event
    agg_sequence = [event[0] if len(
        event) == 1 else event for event in agg_sequence]

    return agg_sequence


In [ ]:
# stays_old = copy.deepcopy(stays)

In [ ]:
indices = [dend['ivl'].index(i) for i in stays]

In [ ]:
a = get_clusters_by_level(0.6, links)
a

In [ ]:
def cluster_events(level, stays, links):
    clusters = get_clusters_by_level(level, links)
    unique_levels = list(set(clusters))
    print(f"clust: {clusters}, ul: {unique_levels}")
   
    cluster_level = copy.deepcopy(level)

    for count, level in enumerate(unique_levels):
        cluster = [i for i, x in enumerate(clusters) if x == level]
        print(f"clust: {cluster}")
        bidx = branch_depths.index(cluster_level)

        sequence_ids = [int(stays[i]) for i in cluster]
        print(f"sids: {sequence_ids}")
       
        alignment_levels = []
        if (cluster_level > 0):
            alignment_levels = list(dict.fromkeys([
                sequence_alignments[bidx - 1][stays.index(s)] for s in sequence_ids]))


        if len(sequence_ids) == 1:
            # base case: sequence need not to be merged
            # print("[INFO] base case, no alignment")
            pass

        elif (len(alignment_levels) == 1 and alignment_levels[0] == -1) or (len(alignment_levels) == 0 and cluster_level == 0):
            # case sequences need to be merged, no prior alignment
            print("[INFO] sequences need to be aligned, no alignment present")

            
            sequence_to_fasta(sequences=[sequences[stays.index(s)] for s in sequence_ids],
                              sequence_ids=sequence_ids, id=f"{cluster_level}-{count}")

            sequences_file_path = f"{output_path + output_folder}sequences-{cluster_level}-{count}.fa"
            base_alignment_file_path = f"{output_path + output_folder}alignment-{cluster_level}-{count}.fasta"

            mafft_align = f"/usr/local/bin/mafft --reorder --anysymbol --maxiterate 0 --retree 1 --6merpair --quiet --thread 4 {sequences_file_path} > {base_alignment_file_path}"
            !$mafft_align
            # os.system(mafft_align)

            alignment = AlignIO.read(base_alignment_file_path, "fasta")
            aggregated_sequence = get_aggregated_sequence(alignment)

            cluster_alignment = {
                'file': base_alignment_file_path,
                'stays': sequence_ids,
                'sequence': aggregated_sequence,
                'alignment': [{'hadm_id': aligned_seq.id, "sequence": [
                    event for event in aligned_seq.seq]} for aligned_seq in alignment]
            }

            save_as_pickle(
                cluster_alignment, f"alignment-info-{number_of_stays}-level-{cluster_level}-count-{count}.p", path=output_path + alignments_output)

            for s in sequence_ids:
                sidx = stays.index(s)
                sequence_alignments[bidx][sidx] = f"{cluster_level}-{count}"

        elif len(alignment_levels) == 1 and alignment_levels[0] != -1:
            # case sequences have already been merged, no action needed
            print("[INFO] have been merged, no action needed")
            for s in sequence_ids:
                sidx = stays.index(s)
                sequence_alignments[bidx][sidx] = sequence_alignments[bidx - 1][sidx]

        else:
            # merging and or alignment needs to happen
            # print("[INFO] merge needed and optional alignment")

            not_aligned_sequences = [
                s for s in sequence_ids if sequence_alignments[bidx - 1][stays.index(s)] == -1]
            
            sequence_to_fasta(sequences=[sequences[stays.index(s)] for s in not_aligned_sequences],
                              sequence_ids=not_aligned_sequences, id=f"{cluster_level}-{count}")

            sequences_file_path = f"{output_path + output_folder}sequences-{cluster_level}-{count}.fa"
            base_alignment_file_path = f"{output_path + output_folder}alignment-{cluster_level}-{count}.fasta"

            # Get alignment files of previous merged
            aligned_sequences = [
                s for s in sequence_ids if sequence_alignments[bidx - 1][stays.index(s)] != -1]

            print(
                f"[INFO] merge needed ({len(aligned_sequences)}) and optional alignment ({len(not_aligned_sequences)})")

            aligned_files = []

            for a in aligned_sequences:
                file_details = sequence_alignments[bidx -
                                                   1][stays.index(a)].split('-')
                aligned_files.append(
                    f"{output_path + output_folder}alignment-{file_details[0]}-{file_details[1]}.fasta")

            aligned_files = list(dict.fromkeys(
                aligned_files))  # Remove duplicates
            table_files = " ".join(aligned_files)
            aligned_files.append(sequences_file_path)
            input_files = " ".join(aligned_files)

            # Create merge table for MAFFT
            merge_table = f"/usr/bin/ruby makemergetable.rb {table_files} > subMSAtable"
            # os.system(merge_table)
            !$merge_table

            # Create input file
            input_command = f"cat {input_files} > inputFile"
            !$input_command

            # os.system(input_command)

            mafft_merge = f"/usr/local/bin/mafft --merge subMSAtable --reorder --anysymbol --maxiterate 0 --retree 1 --6merpair --quiet --thread 4 inputFile > {base_alignment_file_path}"
            # os.system(mafft_merge)
            !$mafft_merge

            alignment = AlignIO.read(base_alignment_file_path, "fasta")
            aggregated_sequence = get_aggregated_sequence(alignment)

            cluster_alignment = {
                'file': base_alignment_file_path,
                'stays': sequence_ids,
                'sequence': aggregated_sequence,
                'alignment': [{'hadm_id': aligned_seq.id, "sequence": [
                    event for event in aligned_seq.seq]} for aligned_seq in alignment]
            }

            save_as_pickle(
                cluster_alignment, f"alignment-info-{number_of_stays}-level-{cluster_level}-count-{count}.p", path=output_path + alignments_output)

            for s in sequence_ids:
                sidx = stays.index(s)
                sequence_alignments[bidx][sidx] = f"{cluster_level}-{count}"

In [ ]:
branch_depths = [-1]
for d in dend['dcoord']:
    branch_depths.append(d[1])
branch_depths = list(dict.fromkeys(branch_depths))
branch_depths.sort()

if not os.path.isdir(output_path + alignments_output):
    os.makedirs(output_path + alignments_output)

sequence_alignments = [[-1] * len(stays) for i in range(len(branch_depths))]

for index, branch_depth in enumerate(tqdm(branch_depths)):
    print(f"Aligning level {index, branch_depth}")
    cluster_events(branch_depth, stays, links )

# save_as_pickle(sequence_alignments, 'alignments_' + str(number_of_stays))


## Create labels for frontend


In [ ]:
careunits = data[data['eventtype'] ==
                 data['eventtype'].unique()[2]]
# values = data[(data['eventtype'] == data['eventtype'].unique()[2])
#               & data['careunit'].isin(careunits)]


label_data = data[['eventtype', 'careunit', 'event_encoded']]
label_data['eventtype'].unique()
# values


In [ ]:

icu_label_data = data[['label', 'value',
                       'valuenum',	'valueuom', 'event_encoded']]
icu_label_data


In [ ]:
labels = []

transfer_label_data = data[['eventtype', 'careunit', 'event_encoded']]
icu_label_data = data[['label', 'value', 'valuenum', 'valueuom']]


for event_type in tqdm(transfer_label_data['eventtype'].unique()):
    # values = data[data['eventtype'] == event_type].drop_duplicates()
    values = transfer_label_data[transfer_label_data['eventtype']
                                 == event_type].drop_duplicates()
    if len(values) == 0:
        continue
    else:
        label = {
            'type': 'Transfer',
            'care_unit': event_type,
            'value_enc': values.iloc[0, 2],
            'values': [{'event_type': v.careunit} for v in values.itertuples(index=True)]
        }

    labels.append(label)

for label in tqdm(icu_label_data['label'].unique()):
    values = icu_label_data[icu_label_data['label'] == label].drop_duplicates()
    if len(values) == 0:
        continue
    else:
        label = {
            'type': 'ICU measurement',
            'measurement': label,
            'value_enc': values.iloc[0, 3],
            'values': [{'value': v.value, 'value_unit': v.valueuom if not pd.isnull(v.valueuom) else -1} for v in values.itertuples(index=True)]
        }
    labels.append(label)

labels


In [ ]:
# labels = []

# total_unique_labels = []

# # total_unique_labels.append(data['label'].unique())
# # total_unique_labels.append(data['careunit'].unique())
# # total_unique_labels.append(data['PerformedProcedureStepDescription'].unique());


# for care_unit in tqdm(data['careunit'].unique()):
#     values = data[data['careunit'] == care_unit].drop_duplicates()
#     label = {
#         'type': 'Transfer',
#         'care_unit': care_unit,
#         'values': [{'event_type': v.eventtype, 'value_enc': v.value_categorical} for v in values.itertuples(index=True)]
#     }
#     labels.append(label)

# # for photo_description in tqdm(data['ProcedureCodeSequence_CodeMeaning'].unique()):
# #     values = data[data['ProcedureCodeSequence_CodeMeaning']
# #                   == photo_description]
# #     label = {
# #         'type': 'Photo',
# #         'viewpoint': photo_description,
# #         'values': [{'view_position': v.ViewPosition, 'patient_orientation': v.PatientOrientationCodeSequence_CodeMeaning, 'value_enc': v.value_categorical} for v in values.itertuples(index=True)]
# #     }
# #     labels.append(label)

# for label in tqdm(data['label'].unique()):
#     values = data[data['label'] == label].drop_duplicates()
#     label = {
#         'type': 'ICU measurement',
#         'measurement': label,
#         'values': [{'value': v.value, 'value_unit': v.valueuom if not pd.isnull(v.valueuom) else -1, 'value_enc': v.value_categorical, } for v in values.itertuples(index=True)]
#     }
#     labels.append(label)


# labels


In [ ]:
changed = False

if 'care_unit' in labels[0] and (type(labels[0]['care_unit']) == 'int' and math.isnan(labels[0]['care_unit'])):
    labels[0]['care_unit'] = 'Not available'
    print(labels[0])
    changed = True

if 'measurement' in labels[len(labels) - 1] and type(labels[len(labels) - 1]) == 'int' and math.isnan(labels[len(labels) - 1]['measurement']):
    labels[len(labels) - 1]['measurement'] = "Not available"
    print(labels[len(labels) - 1])
    changed = True


if changed:
    print('saving...')
    save_as_pickle(labels, 'labels')


## Store data


In [ ]:
save_as_pickle(links, 'links' + file_suffix)
# save_as_pickle(dist_matrix, 'dist_matrix' +
#                file_suffix)  # todo: check if correct
save_as_pickle(stays, 'stays' + file_suffix)
# save_as_pickle(data[data['hadm_id'].isin(stays)], 'events' + file_suffix)
save_as_pickle(sequence_alignments, 'alignments_' + str(number_of_stays))


In [ ]:
def get_event_details(hadm_id, event_index):
    event = data[data['hadm_id'] == int(
        hadm_id)].iloc[[event_index]]

    event = event[event.columns[~event.isnull().all()]]

    return event
